In [44]:
import numpy as np
import pandas as pd

df = pd.read_csv("./heart_attack_prediction_dataset.csv")
df1 = df['Blood Pressure']
type(df1)
arr = df1.to_numpy()
print(df)
arr

     Patient ID  Age     Sex  Cholesterol Blood Pressure  Heart Rate  \
0       BMW7812   67    Male          208         158/88          72   
1       CZE1114   21    Male          389         165/93          98   
2       BNI9906   21  Female          324         174/99          72   
3       JLN3497   84    Male          383        163/100          73   
4       GFO8847   66    Male          318          91/88          93   
...         ...  ...     ...          ...            ...         ...   
8758    MSV9918   60    Male          121          94/76          61   
8759    QSV6764   28  Female          120        157/102          73   
8760    XKA5925   47    Male          250         161/75         105   
8761    EPE6801   36    Male          178         119/67          60   
8762    ZWN9666   25  Female          356         138/67          75   

      Diabetes  Family History  Smoking  Obesity  ...  \
0            0               0        1        0  ...   
1            1       

array(['158/88', '165/93', '174/99', ..., '161/75', '119/67', '138/67'],
      dtype=object)

In [45]:
i = 0
a =[]
b=[]
for i in range(len(arr)):
    x, y= arr[i].split('/')
    a.append(x)
    b.append(y)
    

In [46]:
df = df.drop(columns= ['Blood Pressure'])
df['Systolic BP'] = a
df['Diastolic BP'] = b



In [47]:
df.insert(4, 'Systolic BP', df.pop('Systolic BP'))
df.insert(5, 'Diastolic BP', df.pop('Diastolic BP'))
df

,Patient ID,Age,Sex,Cholesterol,Systolic BP,Diastolic BP,Heart Rate,Diabetes,Family History,Smoking,...,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Attack Risk
0,BMW7812,67,Male,208,158,88,72,0,0,1,...,6.615001,261404,31.251233,286,0,6,Argentina,South America,Southern Hemisphere,0
1,CZE1114,21,Male,389,165,93,98,1,1,1,...,4.963459,285768,27.194973,235,1,7,Canada,North America,Northern Hemisphere,0
2,BNI9906,21,Female,324,174,99,72,1,0,0,...,9.463426,235282,28.176571,587,4,4,France,Europe,Northern Hemisphere,0
3,JLN3497,84,Male,383,163,100,73,1,1,1,...,7.648981,125640,36.464704,378,3,4,Canada,North America,Northern Hemisphere,0
4,GFO8847,66,Male,318,91,88,93,1,1,1,...,1.514821,160555,21.809144,231,1,5,Thailand,Asia,Northern Hemisphere,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8758,MSV9918,60,Male,121,94,76,61,1,1,1,...,10.806373,235420,19.655895,67,7,7,Thailand,Asia,Northern Hemisphere,0
8759,QSV6764,28,Female,120,157,102,73,1,0,0,...,3.833038,217881,23.993866,617,4,9,Canada,North America,Northern Hemisphere,0
8760,XKA5925,47,Male,250,161,75,105,0,1,1,...,2.375214,36998,35.406146,527,4,4,Brazil,South America,Southern Hemisphere,1
8761,EPE6801,36,Male,178,119,67,60,1,0,1,...,0.029104,209943,27.294020,114,2,8,Brazil,South America,Southern Hemisphere,0


In [48]:
df.columns


Index(['Patient ID', 'Age', 'Sex', 'Cholesterol', 'Systolic BP',
       'Diastolic BP', 'Heart Rate', 'Diabetes', 'Family History', 'Smoking',
       'Obesity', 'Alcohol Consumption', 'Exercise Hours Per Week', 'Diet',
       'Previous Heart Problems', 'Medication Use', 'Stress Level',
       'Sedentary Hours Per Day', 'Income', 'BMI', 'Triglycerides',
       'Physical Activity Days Per Week', 'Sleep Hours Per Day', 'Country',
       'Continent', 'Hemisphere', 'Heart Attack Risk'],
      dtype='object')

In [ ]:
print(df.shape)
#checking for not null values
for i in df.columns:
    print(df[i].count())

(8763, 27)
8763
8763
8763
8763
8763
8763
8763
8763
8763
8763
8763
8763
8763
8763
8763
8763
8763
8763
8763
8763
8763
8763
8763
8763
8763
8763
8763
